# PROJECT 1: DATA ANALYSIS PROJECT

**Table of contents**<a id='toc0_'></a>    
- 1. [Aggregate inflation in Denmark](#toc1_)    
  - 1.1. [Basics](#toc1_1_)    
  - 1.2. [Instantaneous inflation](#toc1_2_)    
  - 1.3. [Core inflation](#toc1_3_)    
  - 1.4. [Disaggregated inflation in Denmark](#toc1_4_)   
- 2. [International Comparison](#toc2_)    
  - 2.1. [CPI vs HICP](#toc2_1_)    
  - 2.2. [Across countries](#toc2_2_)    
- 3. [Extension](#toc2_)  

<!-- vscode-jupyter-toc-config
	numbering=true
	anchor=true
	flat=false
	minLevel=2
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

In [ ]:
# Importing packages
import re
import numpy as np
import pandas as pd
from IPython.display import display
import Aflevering as py
import matplotlib.pyplot as plt
from dstapi import DstApi

plt.rcParams.update({'axes.grid':True,'grid.color':'black','grid.alpha':'0.25','grid.linestyle':'--'})
plt.rcParams.update({'font.size': 14})



## 1. <a id='toc1_'></a>[Aggregate inflation in Denmark](#toc0_)

In [ ]:
# Setting up dictionaries for renaming columns and variables for the PRIS113 database
# Making sure that the code is robust with regards to name
columns_dict = {}
columns_dict['TYPE'] = 'variable'
columns_dict['MÅNED'] = 'month'
columns_dict['INDHOLD'] = 'value'

var_dict = {}
var_dict['Forbrugerprisindeks'] = 'CPI'

In [ ]:
# Downloading data from PRIS113 database using dstapi
API = DstApi('PRIS113') 
params = API._define_base_params(language='en') # On english, but can also chose to make it on danish by using 'da'
display(params)
API = API.get_data(params)

In [ ]:
# Renaming columns and variables from the PRIS113 database to the dictionaries defined above
API = API.rename(columns=columns_dict)
for key, value in var_dict.items():
    API.variable.replace(key, value, inplace=True)

API['value'] = pd.to_numeric(API['value'])

API.head()

In [ ]:
# Renaming TID to Time and converting to datetime
API['date'] = pd.to_datetime(API['TID'], format='%YM%m')
API['year'] = API.date.dt.year
API['month'] = API.date.dt.month

# Dropping TID colonen
API = API.drop(columns=['TID'])

# Se datasættet
API.head()

In [ ]:
# Sort after date
API = API.sort_values('date').reset_index(drop=True)
# Verifify that it's sorted
API[['date', 'year', 'month', 'value']].head(10)

### 1.1. <a id='toc1_1_'></a>[Basics](#toc0_)

In [ ]:
# Calculated the consumer price index (CPI), P_t, indexed to 100 on average in 2020
cpi_2020_avg = API[API['year'] == 2020]['value'].mean()
print(f"Gennemsnitlig CPI i 2020: {cpi_2020_avg:.2f}")

# Calculate P_t for all years
API['P_t'] = (API['value'] / cpi_2020_avg) * 100

# Look at the results
API[['date', 'year', 'value', 'P_t']].head(10)
API[['date', 'year', 'value', 'P_t']].tail(10)

In [ ]:
# Calculate yearly average CPI
API_yearly = API.groupby('year')['P_t'].mean().reset_index()

# Plotting the CPI over time
fig, ax = plt.subplots(figsize=(14, 6))
ax.plot(API_yearly['year'], API_yearly['P_t'], marker='o')
ax.set_xlabel('Year')
ax.set_ylabel('CPI (2020=100)')
ax.set_title('CPI in Denmark (yearly average)')
ax.axhline(y=100, color='red', linestyle='--', alpha=0.5, label='2020=100')
ax.grid(True, alpha=0.3)
ax.legend()
plt.tight_layout()
plt.show()

In [ ]:
# Calculate month-to-month inflation rate π_t = value_t / value_{t-1} - 1
API['pi_t'] = API['value'] / API['value'].shift(1) - 1

# Look at the results
API[['date', 'year', 'value', 'pi_t']].tail(20)

In [ ]:
# Filter from 2010
API_2010 = API[API['year'] >= 2010]

# Plot as a barchart
fig, ax = plt.subplots(figsize=(14, 6))
ax.bar(API_2010['date'], API_2010['pi_t'] * 100, width=20, alpha=0.7)
ax.set_xlabel('Year')
ax.set_ylabel('Month to month inflation(%)')
ax.set_title('Month to month inflaiton in Denmark(2010-)')
ax.axhline(y=0, color='black', linestyle='-', linewidth=0.5)
ax.grid(True, alpha=0.3, axis='y')
plt.tight_layout()
plt.show()

In [ ]:
# Calculate 12-month inflation rate π_t^12 = value_t / value_{t-12} - 1
API['pi_t_12'] = API['value'] / API['value'].shift(12) - 1

# Se resultatet
API[['date', 'year', 'value', 'pi_t_12']].tail(20)

In [ ]:
# Filter from 2010
API_2010 = API[API['year'] >= 2010]

# Plot as a barchart
fig, ax = plt.subplots(figsize=(14, 6))
ax.bar(API_2010['date'], API_2010['pi_t_12'] * 100, width=20, alpha=0.7)
ax.set_xlabel('Year')
ax.set_ylabel('12 months inflation (%)')
ax.set_title('12 month inflation in Denmark (2010-)')
ax.axhline(y=0, color='black', linestyle='-', linewidth=0.5)
ax.grid(True, alpha=0.3, axis='y')
ax.legend()
plt.tight_layout()
plt.show()

**When did the post-pandemic inflation surge end in Denmark?**

The post-pandemic inflation surge in Denmark peaked in late 2022 to early 2023, reaching approximately 10% on a 12-month basis. After this peak, inflation declined steadily throughout 2023. By late 2023 and into early 2024, inflation had returned to around 2% or below, comparable to pre-pandemic levels. The month-to-month data shows that price volatility also normalized during this period. While inflation rates returned to target levels, the cumulative CPI index shows that the overall price level remained permanently elevated from the surge.

### 1.2. <a id='toc1_2_'></a>[Instantaneous inflation](#toc0_)

In [ ]:
# 1. Setting start values and computing kappa(k,α)
T = 11
k_values = np.arange(0, 12)
alpha_values = [0, 1, 2, 3]

def compute_kappa(k, alpha, T=11):
    """Beregn vægtfunktion κ(k, α)"""
    if alpha == 0:
        return 1/12  # Lige vægtning når α=0
    numerator = (T - k)**alpha
    denominator = sum((T - j)**alpha for j in range(12))
    return numerator / denominator

# Plot κ(k,α) for k = 0,1,...,11 og α = 0,1,2,3
fig, ax = plt.subplots(figsize=(10, 6))
for alpha in alpha_values:
    kappas = [compute_kappa(k, alpha) for k in k_values]
    ax.plot(k_values, kappas, marker='', label=f'α={alpha}')

ax.set_xlabel('k (months back)')
ax.set_ylabel('κ(k,α)')
ax.set_title('Weight function κ(k,α) - higher α gives more weight to recent months')
ax.legend()
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
# 2. Compute π_t^{12,α} using rolling method
def compute_instantaneous_inflation_rolling(pi_t, alpha):
    """
    Compute π_t^{12,α} using rolling window
    pi_t: monthly inflation rates (as decimals)
    """
    def weighted_inflation(window):
        if len(window) < 12:
            return np.nan
        # Window is ordered oldest to newest, reverse to match k indexing
        # where k=0 is most recent
        window_reversed = window[::-1]
        # Compute weighted geometric mean: ∏(1 + π_t-k)^κ(k,α) - 1
        weights = [compute_kappa(k, alpha) for k in range(12)]
        product = np.prod((1 + window_reversed) ** weights)
        return product - 1
    
    return pi_t.rolling(window=12).apply(weighted_inflation, raw=True)

# Calculate for all α values
for alpha in alpha_values:
    API[f'pi_inst_alpha_{alpha}'] = compute_instantaneous_inflation_rolling(API['pi_t'], alpha)

# Verify that at α=0, it gives the same result as standard 12-month inflation
print("Comparison α=0 vs standard 12-month inflation:")
comparison = API[['date', 'pi_t_12', 'pi_inst_alpha_0']].tail(10)
comparison['difference'] = comparison['pi_t_12'] - comparison['pi_inst_alpha_0']
print(comparison)


In [ ]:
# 3. Plot siden 2019
API_2019 = API[API['year'] >= 2019]

fig, ax = plt.subplots(figsize=(14, 7))
for alpha in alpha_values:
    ax.plot(API_2019['date'], API_2019[f'pi_inst_alpha_{alpha}'] * 100, 
            label=f'α={alpha}', linewidth=2)

ax.set_xlabel('År')
ax.set_ylabel('Instantaneous inflation (%)')
ax.set_title('Instantaneous inflation π_t^{12,α} i Danmark (2019-)')
ax.axhline(y=2, color='red', linestyle='--', linewidth=1, alpha=0.7, label='2% målsætning')
ax.grid(True, alpha=0.3)
ax.legend()
plt.tight_layout()
plt.show()

# Find hvornår inflationen toppede for forskellige α
print("\nHvornår toppede inflationen for forskellige α:")
for alpha in alpha_values:
    post_2020 = API[API['year'] >= 2020]
    peak_idx = post_2020[f'pi_inst_alpha_{alpha}'].idxmax()
    peak_data = post_2020.loc[peak_idx]
    print(f"α={alpha}: {peak_data['date'].strftime('%Y-%m')}, {peak_data[f'pi_inst_alpha_{alpha}']*100:.2f}%")

The standard measure (α = 0) peaks in late 2022 at 0.8%. The high-α measures
(α = 2, 3) peak earlier in mid-2022 at 1.0% and decline sharply thereafter—reaching
negative territory by late 2023 while α = 0 remains elevated.
This suggests high-α measures can signal turning points months ahead of standard
inflation, acting as an early warning system, while α = 0 provides a more stable
structural view of inflationary periods.

### 1.3. <a id='toc1_2_'></a>[Core inflation](#toc0_)

In [ ]:
# Get data from PRIS111 (Consumer Price Index - detailed)
pris111_api = DstApi('PRIS111')

params = pris111_api._define_base_params(language='en')
display(params)
pris111 = pris111_api.get_data(params)

#see unique values in various columns
print(pris111['ENHED'].unique()) 
print(pris111['VAREGR'].unique())

In [ ]:
# Define filters (relevant groups and unit)
target_unit = "Percentage change compared to same month the year before (per cent)"
groups = [
    "00 Consumer price index, total",
    "15.1 Overall Consumer Price Index excl. energy",
    "14.1 Overall Consumer Price Index - excl. energy and unprocessed food (core inflation)",
]

# using the filters
df = pris111.loc[
    (pris111["ENHED"] == target_unit) & (pris111["VAREGR"].isin(groups)),
    ["TID", "VAREGR", "INDHOLD"]
].copy()

# Convert to numeric & datetime 
df["INDHOLD"] = pd.to_numeric(df["INDHOLD"], errors="coerce")
df["date"] = pd.to_datetime(df["TID"].str.replace("M", "-"), format="%Y-%m", errors="coerce")

# --- Aggregate if duplicates ---
#df = df.groupby(["date", "VAREGR"], as_index=False)["INDHOLD"].mean()

# Pivot to wide format 
wide = (
    df.pivot(index="date", columns="VAREGR", values="INDHOLD")
      .sort_index()
      .loc["2019-01-01":]
)

# Rename for cleaner legend labels ---
name_map = {
    "00 Consumer price index, total": "CPI (Total)",
    "15.1 Overall Consumer Price Index excl. energy": "CPI excl. energy",
    "14.1 Overall Consumer Price Index - excl. energy and unprocessed food (core inflation)": "Core inflation",
}
wide = wide.rename(columns=name_map) #applying the rename

# Plot 
fig, ax = plt.subplots(figsize=(12, 6))
wide.plot(ax=ax, linewidth=2)
ax.axhline(0, color="black", linewidth=1)
ax.set_title("Denmark: Year-over-Year Inflation Rates (since 2019)", fontsize=14)
ax.set_xlabel("Date")
ax.set_ylabel("Year-over-Year Change (%)")
ax.legend(title="Measure", loc="upper left", frameon=False)
ax.grid(True, axis="y", linestyle="--", alpha=0.7)
plt.tight_layout()
plt.show()

**Comments on the figure** :

Using the PRIS111 table from Statistics Denmark, we can decompose inflation into several distinct groups. This allows us to separate the effects of energy and food prices from what is often referred to as core inflation.

The figure above displays the annual growth rate of overall inflation, inflation excluding energy, and inflation excluding both energy and food. The figure shows that particular energy has been a driving factor in the large inflation we experienced towards the end of 2022. 

### 1.4. <a id='toc1_4_'></a>[Disaggregated inflation in Denmark](#toc0_)

In [ ]:
# Inspect the category levels
levels = DstApi('PRIS111').variable_levels('VAREGR', language='en')

# Print all categories
for row in levels.itertuples():
    print(row.id, "-", row.text)

In [ ]:
# We extract the numeric part from 'text' (before the first space)
levels['numcode'] = levels['text'].str.extract(r'^([\d\.]+)')

# Remove dots and count digits
levels['numcode_clean'] = levels['numcode'].str.replace('.', '', regex=False)

# Keep only those with 4 digits
filtered = levels[levels['numcode_clean'].str.len() == 4]

# Print the result
for row in filtered.itertuples():
    print(row.id, "-", row.text)

In [ ]:
# --- metadata for canonical names at 01.1.1 level (optional but nice) ---
levels = DstApi('PRIS111').variable_levels('VAREGR', language='en').copy()
names_3seg = (
    levels['text']
      .str.extract(r'^(?P<code3>\d{2}\.\d\.\d)\s+(?P<name3>.+)$')
      .dropna()
      .drop_duplicates()
)

# --- prices ---
prices = pris111[pris111['ENHED'] == 'Index'].copy()

# Keep ONLY rows where VAREGR is EXACTLY three segments: "01.1.1 <name>"
exact_3seg = prices['VAREGR'].str.match(r'^\d{2}\.\d\.\d\s')
prices = prices.loc[exact_3seg].copy()

# Extract code + (raw) name from VAREGR
m = prices['VAREGR'].str.extract(r'^(?P<code3>\d{2}\.\d\.\d)\s+(?P<name_raw>.+)$')
prices = pd.concat([prices, m], axis=1)   # avoids the "columns overlap" join error

# Parse value and date
prices['INDHOLD'] = pd.to_numeric(prices['INDHOLD'].astype(str).str.replace(',', '.'), errors='coerce')
prices['date'] = pd.to_datetime(prices['TID'].str.replace('M', '-'), format='%Y-%m', errors='coerce')

# Drop rows without value/date
prices = prices.dropna(subset=['INDHOLD', 'date'])

# Attach canonical name (if available)
prices = prices.merge(names_3seg, on='code3', how='left')
prices['name'] = prices['name3'].fillna(prices['name_raw'])
prices = prices.drop(columns=['name3'])

# Sanity checks (should be zero)
#   multiple rows per (date, code3)?
dupes = prices.duplicated(['date', 'code3'], keep=False).sum()
#   any NaNs left in INDHOLD?
n_nans = prices['INDHOLD'].isna().sum()

print(f"Duplicated (date, code3) rows: {dupes}")
print(f"NaNs in INDHOLD: {n_nans}")

# Sort and preview
prices = prices.sort_values(['date', 'code3'])
print(prices[['date', 'code3', 'name', 'INDHOLD']].head(12))

In [ ]:
# Compute 12-month percent change (YoY inflation)
prices['infl_12m'] = prices.groupby('code3')['INDHOLD'].pct_change(12) * 100

# Drop rows without inflation value
prices = prices.dropna(subset=['infl_12m'])

#preview 
print(prices[['date','code3','name','INDHOLD','infl_12m']].head(12))


In [ ]:
# Group by date and compute distribution percentiles
summary = (
    prices.groupby('date')['infl_12m']
          .agg(
              p25=lambda x: x.quantile(0.25),
              p50=lambda x: x.quantile(0.50),
              p75=lambda x: x.quantile(0.75)
          )
          .reset_index()
)


fig, ax = plt.subplots(figsize=(12, 6))

ax.plot(summary['date'], summary['p25'], linestyle='--', label='25th percentile')
ax.plot(summary['date'], summary['p50'], linewidth=2, label='Median (50th)')
ax.plot(summary['date'], summary['p75'], linestyle='--', label='75th percentile')

ax.set_title('Distribution of 12-Month Inflation (4-digit CPI categories)')
ax.set_xlabel('Month')
ax.set_ylabel('Inflation rate (%)')
ax.legend()
ax.grid(True, axis='y', alpha=0.7)
plt.tight_layout()
plt.show()

***1.4.3 - Plot histogram***

In [ ]:
# Ensure we only use the relevant window
start, end = "2020-08-01", "2025-08-01"

# Get price levels for those months
df_2020 = prices.loc[prices['date'] == start, ['code3', 'INDHOLD']].rename(columns={'INDHOLD': 'p_2020'})
df_2025 = prices.loc[prices['date'] == end,   ['code3', 'INDHOLD']].rename(columns={'INDHOLD': 'p_2025'})

# Merge and compute percent change
change = (
    df_2020.merge(df_2025, on='code3', how='inner')
           .assign(pct_change=lambda d: (d['p_2025'] / d['p_2020'] - 1) * 100)
           .dropna(subset=['pct_change'])
)


In [ ]:
fig, ax = plt.subplots(figsize=(10, 6))
ax.hist(change['pct_change'], bins=40, color='skyblue', edgecolor='black', alpha=0.8)

ax.set_title('Distribution of 5-Year Price Changes (Aug 2020 – Aug 2025)\n4-digit CPI categories')
ax.set_xlabel('Percent change in price (%)')
ax.set_ylabel('Number of product categories')
ax.grid(axis='y', alpha=0.5)

plt.tight_layout()
plt.show()


DET FORKERT - RET til din beskrivelse af 5 års
The histogram shows that most categories cluster around 2 or 3 percent, which corresponds somewhat to the overall inflation level. However there are outliers, especially on the right hand side of 0, where some goods have seen a strong increase.

The relatively narrow shape of the histogram implies low overall dispersion, meaning inflation in August 2025 was more balanced across categories than during earlier more volatile periods.

In [ ]:
# Get the last available month 
latest_month = prices['date'].max()

print(prices.columns.tolist())

latest = prices.loc[prices['date'] == latest_month]

In [ ]:
# Sort by inflation rate
top10 = latest.nlargest(10, 'infl_12m')
bottom10 = latest.nsmallest(10, 'infl_12m')

print("Top 10 categories (highest inflation):")
print(top10[['code3', 'name', 'infl_12m']])

print("\nBottom 10 categories (lowest inflation):")
print(bottom10[['code3', 'name', 'infl_12m']])

In [ ]:
# Get the latest month (should be Aug 2025)
latest_month = prices['date'].max()

# Extract 12-month inflation rates for that month
latest = (
    prices.loc[prices['date'] == latest_month, ['code3', 'name', 'infl_12m']]
    .dropna(subset=['infl_12m'])
)

# Rank categories by inflation
top10 = latest.nlargest(10, 'infl_12m')
bottom10 = latest.nsmallest(10, 'infl_12m')

# Combine and label for plotting
ranked = pd.concat([top10, bottom10])
ranked['category'] = ranked['code3'] + ' ' + ranked['name']

# Sort for plotting (lowest at bottom, highest at top)
ranked_sorted = ranked.sort_values('infl_12m')

# Bar chart
fig, ax = plt.subplots(figsize=(8, 6))
ax.barh(ranked_sorted['category'], ranked_sorted['infl_12m'], color='slateblue')
ax.axvline(0, color='black', linewidth=1)
ax.set_title('Top 10 and Bottom 10 12-Month Inflation Categories (Aug 2025)')
ax.set_xlabel('12-Month Inflation Rate (%)')
ax.grid(True, axis='x', alpha=0.5)
plt.tight_layout()
plt.show()

## 2. <a id='toc2_'></a>[International Comparison](#toc0_)

### 2.1. <a id='toc2_1_'></a>[CPI vs HICP](#toc0_)

In [ ]:
pip install fredapi

In [ ]:
pip install pandas_datareader

In [ ]:
from fredapi import Fred
from pandas_datareader import data as pdr
from IPython.display import display

In [ ]:
# Download CPI data from Statistics Denmark
print("Download CPI from Statistics Denmark")
api = DstApi('PRIS113')
params = api._define_base_params(language='en')
data = api.get_data(params)

# print first five rows to track data
print(f"Number of rows downloaded: {len(data)}")
display(data.head())

# keep only 2010 and onward
data = data[data['TID'] >= '2010M01']
print(f"\nAfter filtering (from 2010): {len(data)} observations")
display(data.head())

# rename to english (standardized names)
rename_map = {
    'TID': 'time', 'time': 'time',
    'INDHOLD': 'value', 'value': 'value',
    'TYPE': 'type', 'type': 'type'
}
for k, v in rename_map.items():
    if k in data.columns:
        data = data.rename(columns={k: v})

# date format, sort and rename
df_cpi = data[['time', 'value']].copy()
df_cpi['date'] = pd.to_datetime(df_cpi['time'], format='%YM%m')
df_cpi = df_cpi.sort_values('date')
df_cpi = df_cpi.rename(columns={'value': 'CPI'})

#print to keep track
print("\nCPI data after cleaning:")
display(df_cpi.head())
print(f"CPI date range: {df_cpi['date'].min()} → {df_cpi['date'].max()}")

# get HICP data from FRED
print("\nDownload HICP from FRED")
from fredapi import Fred
fred = Fred(api_key="60c118ce211cbc54a03242bbf7dc7c19")
hicp = fred.get_series('CP0000DKM086NEST').reset_index()
hicp.columns = ['date', 'HICP']

#keep track
print(f"HICP date range: {hicp['date'].min()} → {hicp['date'].max()}")
print(f"Number of rows downloaded from FRED: {len(hicp)}")
display(hicp.head())

# merging DST and FRED, and print
print("\nMerging CPI and HICP")
merged = pd.merge(df_cpi, hicp, on='date', how='inner')
merged = merged[merged['date'] >= '2010-01-01']
print(f"Number of overlapping observations (since 2010): {len(merged)}")
print(f"Columns: {list(merged.columns)}")
print(f"Date range: {merged['date'].min().strftime('%Y-%m')} → {merged['date'].max().strftime('%Y-%m')}")
print("\nFull merged dataset:")
display(merged)

# normalize such that index (2019=100), for asignmnet 2.2
print("\nNormalizing both series to average = 100 in 2019")

# keep sure that columns is num
merged['CPI'] = pd.to_numeric(merged['CPI'], errors='coerce')
merged['HICP'] = pd.to_numeric(merged['HICP'], errors='coerce')

# average CPI and HICP in 2019 
base_period = merged.loc[merged['date'].dt.year == 2019, ['CPI', 'HICP']].mean()
print("Average values in 2019:")
display(base_period.to_frame().T)

# normalized series
merged['CPI_norm'] = merged['CPI'] / base_period['CPI'] * 100
merged['HICP_norm'] = merged['HICP'] / base_period['HICP'] * 100

print("\nMerged data 2019=100:")
display(merged.head())

# Plot CPI vs HICP
print("\nPlot CPI vs HICP")
plt.figure(figsize=(10,6))
plt.plot(merged['date'], merged['CPI_norm'], label='CPI (Statistics Denmark)')
plt.plot(merged['date'], merged['HICP_norm'], label='HICP (FRED)', linestyle='--')
plt.title('CPI vs HICP in Denmark (Indexed to 2019 = 100)')
plt.ylabel('Index (2019 = 100)')
plt.xlabel('Date')
plt.legend()
plt.grid(True)
plt.show()


When comparing the CPI and the HICP, it can be noted that they are closely related. They both measure inflation, and can in fact be compared directly, although they are not perfectly identical. According to Statistics Denmark, they both use about the same data sources and methods. The main difference between them is how they include owner-occupied dwellings and private hospitals. CPI includes them, while HICP excludes them such that the data is comparable between countries. As seen in the figure above, the two have the same trend and level. There are minor differences because of the difference in what they cover.
It's indexed to 2019 because of it's use section 2.2

### 2.2. <a id='toc2_2_'></a>[Across countries](#toc0_)

***Task 2.2.1 and 2.2.2***

In [ ]:
# Connecting to FRED  
fred = Fred(api_key="60c118ce211cbc54a03242bbf7dc7c19")

# Defining FRED series IDs  
fred_ids = {
    "Denmark":        "CP0000DKM086NEST",   # HICP (monthly)
    "Austria":        "CP0000ATM086NEST",   # HICP
    "Euro Area":      "CP0000EZ19M086NEST", # HICP (EA19 – Euro area)
    "United States":  "CP0000USM086NEST"    # HICP (for US in FRED; alternatively use CPIAUCSL)
}
# Fetching data and combining into a single DataFrame (long format)
dfs = []
for country, sid in fred_ids.items():
    s = fred.get_series(sid, observation_start="2019-01-01")  # returns pandas.Series
    df = s.rename("value").to_frame().reset_index().rename(columns={"index": "date"})
    df["country"] = country
    dfs.append(df)

hicp_long = pd.concat(dfs, ignore_index=True).sort_values(["country", "date"]).reset_index(drop=True)

# Converting to wide format 
hicp_wide = hicp_long.pivot(index="date", columns="country", values="value").sort_index()

# Normalizing to 2019 = 100
def norm_2019(g):
    base = g.loc[g.index.year == 2019].mean()   # average for 2019
    return g / base * 100

hicp_2019 = hicp_wide.groupby(axis=1, level=0).apply(norm_2019)
hicp_2019 = hicp_2019.loc[hicp_2019.index >= "2019-01-01"]

# Computing 12-month inflation (YoY) 
hicp_yoy = hicp_wide.pct_change(12) * 100
hicp_yoy = hicp_yoy.loc[hicp_yoy.index >= "2019-01-01"]

# Plot: Indices (2019 = 100)  
plt.figure(figsize=(10,6))
for col in ["Denmark", "Austria", "Euro Area", "United States"]:
    plt.plot(hicp_2019.index, hicp_2019[col], label=col)
plt.title("HICP (All items), normalized to 2019 = 100")
plt.xlabel("Date"); plt.ylabel("Index (2019 = 100)")
plt.legend(); plt.grid(True)
plt.show()

# Plot: 12-month inflation (YoY)  
plt.figure(figsize=(10,6))
for col in ["Denmark", "Austria", "Euro Area", "United States"]:
    plt.plot(hicp_yoy.index, hicp_yoy[col], label=col)
plt.title("12-month inflation rate (YoY)")
plt.xlabel("Date"); plt.ylabel("Percent")
plt.legend(); plt.grid(True)
plt.show()

**Note to the first figure:** 
The indices are normalized so that the average level during 2019 equals 100. Individual months in 2019 may deviate slightly above or below 100, but the overall mean for the year is set as the baseline.

***Task 2.2.3***

In [ ]:
# Prepare yearly statistics 
inflation = hicp_yoy.copy()
inflation['year'] = inflation.index.year

# Min, max, mean per country and year
stats_by_year = (
    inflation.groupby('year')[["Denmark","Austria","Euro Area","United States"]]
    .agg(['min','max','mean'])
    .round(2)
)

print("Yearly min, max, and mean 12-month inflation rates for each country")
display(stats_by_year)

# Overall (since 2019)
overall_stats = (
    hicp_yoy[["Denmark","Austria","Euro Area","United States"]]
    .agg(['min','max','mean'])
    .round(2)
)

print("\nOverall min, max, and mean since 2019")
display(overall_stats)

**Cross-Country Differences in Inflation Since 2019**

The figure and summary statistics reveal that inflation across Denmark, Austria, the Euro Area, and the United States followed broadly similar paths, but with notable differences in timing and magnitude.

In 2020, inflation briefly declined in all economies due to the COVID-19 lockdowns. Average inflation rates were close to zero or slightly positive, ranging from –0,7 % in the United States to around 1 % in Austria.

During 2021–2022, inflation accelerated sharply in response to supply-chain disruptions and energy price shocks. Austria experienced the highest peak, exceeding 11,5 %. All countries inflation rate peaked in 2022 and United States inflation fell more quickly afterward.

From 2023 onward, inflation declined across all regions as energy prices normalized and monetary policy tightened. By 2024–2025, inflation had converged toward 1–3 % in most countries, consistent with central-bank targets.  

Overall, the inflation paths were synchronized but not identical: the U.S. inflation spike occurred earlier and subsided faster, while Austria and the Euro Area faced more persistent price pressures through late 2023.

## 3. <a id='toc3_'></a>[Extension](#toc0_)

In [ ]:
# --- Connect to FRED ---
from fredapi import Fred
import pandas as pd
import matplotlib.pyplot as plt

fred = Fred(api_key="60c118ce211cbc54a03242bbf7dc7c19")

# --- Define FRED series IDs ---
series = {
    # Unemployment rate (%)
    "Denmark":        {"unemp": "LRHUTTTTDKM156S"},  # 15 years and above
    "Austria":        {"unemp": "LRHUTTTTATM156S"},  # 15 years and above
    "Euro Area":      {"unemp": "LRHUTTTTEZM156S"},  # Euro area (EZM)
    "United States":  {"unemp": "UNRATE"},           # 16 years and above

    # Real GDP (index or national currency, SA)
    "DK_GDP": "CLVMNACSCAB1GQDK",    # Denmark
    "AT_GDP": "CLVMNACSCAB1GQAT",    # Austria
    "EA_GDP": "CLVMNACSCAB1GQEA19",  # Euro Area
    "US_GDP": "GDPC1",               # US (billions, chained)

    # Government expenditure (du har tjekket ID'erne; jeg ændrer dem ikke)
    "DK_GOV": "NAEXKP03DKQ189S",
    "AT_GOV": "NAEXKP03ATQ189S",
    "US_GOV": "FGEXPND"
}

# --- Fetch unemployment (monthly) ---
u_dfs = []
for country in ["Denmark", "Austria", "Euro Area", "United States"]:
    sid = series[country]["unemp"]
    s = fred.get_series(sid, observation_start="2019-01-01").rename("value").to_frame().reset_index()
    s["country"] = country
    u_dfs.append(s)

unemp_long = pd.concat(u_dfs).sort_values(["country", "index"])
unemp_wide = unemp_long.pivot(index="index", columns="country", values="value")
unemp_wide = unemp_wide.loc[unemp_wide.index >= "2019-01-01"]

# --- Helper to flatten possible MultiIndex columns from concat(dict=...) ---
def flatten_columns(df):
    if isinstance(df.columns, pd.MultiIndex):
        if df.columns.nlevels > 1:
            df.columns = df.columns.get_level_values(0)
        else:
            df.columns = [str(c) for c in df.columns]
    else:
        df.columns = [str(c) for c in df.columns]
    return df

# --- Fetch GDP (quarterly) ---
gdp_ids = {"Denmark": "DK_GDP", "Austria": "AT_GDP", "Euro Area": "EA_GDP", "United States": "US_GDP"}
gdp_wide = pd.concat(
    {k: fred.get_series(series[v], observation_start="2019-01-01") for k, v in gdp_ids.items()},
    axis=1
)
gdp_wide.index = pd.to_datetime(gdp_wide.index)
gdp_wide = flatten_columns(gdp_wide)

# --- Fetch Government Expenditure (quarterly, DK/AT/US) ---
gov_ids = {"Denmark": "DK_GOV", "Austria": "AT_GOV", "United States": "US_GOV"}
gov_wide = pd.concat(
    {k: fred.get_series(series[v], observation_start="2019-01-01") for k, v in gov_ids.items()},
    axis=1
)
gov_wide.index = pd.to_datetime(gov_wide.index)
gov_wide = flatten_columns(gov_wide)

# --- Normalize GDP & Government Expenditure to 2019 = 100 ---
def norm_2019(g):
    base = g.loc[g.index.year == 2019].mean()
    return g / base * 100

gdp_norm = gdp_wide.apply(norm_2019)
gov_norm = gov_wide.apply(norm_2019)

# --- Plot Unemployment ---
plt.figure(figsize=(10,6))
for col in unemp_wide.columns:
    plt.plot(unemp_wide.index, unemp_wide[col], label=col)
plt.title("Unemployment Rate (%) — 2019–2025")
plt.xlabel("Date")
plt.ylabel("Percent")
plt.legend(title="Country")
plt.grid(True)
plt.tight_layout()
plt.show()

# --- Plot Real GDP (2019 = 100) ---
plt.figure(figsize=(10,6))
for col in gdp_norm.columns:
    plt.plot(gdp_norm.index, gdp_norm[col], label=col)
plt.title("Real GDP, normalized to 2019 = 100")
plt.xlabel("Date")
plt.ylabel("Index (2019 = 100)")
plt.legend(title="Country")
plt.grid(True)
plt.tight_layout()
plt.show()

gov_norm_pre23 = gov_norm.loc[gov_norm.index < "2023-01-01"]

# --- Plot Government Expenditures (2019 = 100) ---
plt.figure(figsize=(10,6))
for col in gov_norm_pre23.columns:
    plt.plot(gov_norm_pre23.index, gov_norm_pre23[col], label=col)
plt.title("Government expenditures, normalized to 2019 = 100")
plt.xlabel("Date")
plt.ylabel("Index (2019 = 100)")
plt.legend(title="Country")
plt.grid(True)
plt.tight_layout()
plt.show()

**Note**: In Figure 1, there is no data for the Euro Area after 2023. In Figure 3, there is no data for the Euro Area, and for the other countries data are only available up to around mid-2022.

Also, please note that the U.S. government expenditure series (FGEXPND) is reported in current prices, while the series for Denmark and Austria are in constant (real) prices.
This means the U.S. values are not directly comparable to the European data, as they include the effects of inflation.
⚠️ To be corrected before the exam: replace the U.S. series with a real (chained-volume) government expenditure series.